# openai 라이브러리 설치

In [1]:
%pip install openai==0.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 644.4 kB/s eta 0:00:00


# 필요 라이브러리 설치

In [2]:
%pip install --upgrade tenacity
%pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 33.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
astropy 5.3.4 requires numpy<2,>=1.21, but you have numpy 2.0.0 which is incompatible.
cudf-cu12 24.4.1 requires numpy<2.0a0,>=1.23, but you have numpy 2.0.0 which is incompatible.
cupy-cuda12x 12.2.0 requires numpy<1.27,>=1.20, but you have numpy 2.0.0 which is incompatible.
ibis-framework 8.0.0 requires numpy<2,>=1, but you have numpy 2.0.0 which is incompatible.
numba 0.58.1 requires numpy<1.27,>=1.22, but you have numpy 2.0.0 which is incompatible.
rmm-cu12 24.4.0 requires numpy<2.0a0,>=1.23, but you have numpy 2.0.0 which is incompatible.
scipy 1.11.4 requires numpy<1.28.0,>=1.21.6

In [4]:
import json
import os
import pandas as pd
from pprint import pprint

# openai 사용 선언과 키 설정

In [5]:
import openai

openai.api_key = ""

# 데이터 읽기

In [20]:
!wget https://raw.githubusercontent.com/dhrim/2024_kangnam_hallym_workshop/main/material/data/cookbook_recipes.csv -O cookbook_recipes.csv


--2024-07-13 23:17:24--  https://raw.githubusercontent.com/dhrim/2024_kangnam_hallym_workshop/main/material/data/cookbook_recipes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63471 (62K) [text/plain]
Saving to: ‘cookbook_recipes.csv’

cookbook_recipes.cs 100%[===================>]  61.98K  --.-KB/s    in 0.02s   

2024-07-13 23:17:25 (2.73 MB/s) - ‘cookbook_recipes.csv’ saved [63471/63471]



In [7]:
recipe_df = pd.read_csv("cookbook_recipes.csv")

recipe_df.head()

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


# 학습 데이터 준비

학습 데이터는 다음과 같은 형태로 준비한다.

```
[
  {'role': 'system', 'content': 'GPT의 역할을 명시한다.'},
  {'role': 'user', 'content': 'GPT에게 전달할 입력 값'},
  {'role': 'assistant', 'content': 'GPT가 입력 데이터를 받아서 출력할 값'}
 ]
```

본 실습에서는 조리법을 입력으로 하여 재료성분을 출력하려 한다. <br>
GPT는 재료 성분을 추출하는 역할을 하고, 입력으로 읽어들인 데이터에서 title과 ingredient 상세로 하고, 출력으로 재료성분 리스트를 출력하게 한다.<br>


실제 학습 데이터는 다음과 같다.

```
[
  {'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},

  {'role': 'user', 'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},

  {'role': 'assistant', 'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}
 ]
```

다음 코드는 파일에서 읽은 데이터를 학습에 필요한 데이터 포멧으로 변경하는 함수이다.

In [8]:
training_data = []

system_message = "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."

def create_user_message(row):
    return f"""Title: {row['title']}\n\nIngredients: {row['ingredients']}\n\nGeneric ingredients: """

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    messages.append({"role": "assistant", "content": row["NER"]})

    return {"messages": messages}

pprint(prepare_example_conversation(recipe_df.iloc[0]))

{'messages': [{'content': 'You are a helpful recipe assistant. You are to '
                          'extract the generic ingredients from each of the '
                          'recipes provided.',
               'role': 'system'},
              {'content': 'Title: No-Bake Nut Cookies\n'
                          '\n'
                          'Ingredients: ["1 c. firmly packed brown sugar", '
                          '"1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 '
                          'c. broken nuts (pecans)", "2 Tbsp. butter or '
                          'margarine", "3 1/2 c. bite size shredded rice '
                          'biscuits"]\n'
                          '\n'
                          'Generic ingredients: ',
               'role': 'user'},
              {'content': '["brown sugar", "milk", "vanilla", "nuts", '
                          '"butter", "bite size shredded rice biscuits"]',
               'role': 'assistant'}]}


# 실제 데이터 생성

학습 데이터를 생성한다. 80개

In [9]:
training_df = recipe_df.loc[0:80]

training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: Jewell Ball\'S Chicken\n\nIngredients: ["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["bee

검증 데이터를 생성한다. 20개

In [10]:
validation_df = recipe_df.loc[81:90]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

# 데이터 파일로 쓰기

실제 학습은 OpenAI에 업로드 하여 이루어 진다. 업로드하도록 파일로 저장한다.

In [12]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

training_file_name = "recipe_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "recipe_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

# 파일 업로드

In [14]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-eAdCpJLotTDQuJiN3E2F7LRp
Validation file ID: file-6es00d3YhuA6LcdEulUi98Xm


# 학습

기존 학습된 모델에 추가 학습하는 것을 fine tuning이라 한다.

In [15]:
response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="recipe-ner",
)

job_id = response["id"]

print("Job ID:", response["id"])
print("Status:", response["status"])

Job ID: ftjob-ROcGxCBufJJdp5iGnYGIbW6I
Status: validating_files


# 학습 상태 조회

In [21]:
response = openai.FineTuningJob.retrieve(job_id)

print("Job ID:", response["id"])
print("Status:", response["status"])
print("Trained Tokens:", response["trained_tokens"])


Job ID: ftjob-ROcGxCBufJJdp5iGnYGIbW6I
Status: running
Trained Tokens: None


In [24]:
response = openai.FineTuningJob.retrieve(job_id)

print("Job ID:", response["id"])
print("Status:", response["status"])
print("Trained Tokens:", response["trained_tokens"])


Job ID: ftjob-ROcGxCBufJJdp5iGnYGIbW6I
Status: succeeded
Trained Tokens: 31812


# 학습 진행 내역 조회

In [25]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Step 198/243: training loss=0.00
Step 199/243: training loss=0.00
Step 200/243: training loss=0.54, validation loss=1.07
Step 201/243: training loss=0.00
Step 202/243: training loss=0.00
Step 203/243: training loss=0.00
Step 204/243: training loss=0.00
Step 205/243: training loss=0.42
Step 206/243: training loss=0.00
Step 207/243: training loss=0.09
Step 208/243: training loss=0.00
Step 209/243: training loss=0.00
Step 210/243: training loss=0.00, validation loss=0.52
Step 211/243: training loss=0.00
Step 212/243: training loss=0.00
Step 213/243: training loss=0.00
Step 214/243: training loss=0.00
Step 215/243: training loss=0.00
Step 216/243: training loss=0.00
Step 217/243: training loss=0.00
Step 218/243: training loss=0.00
Step 219/243: training loss=0.00
Step 220/243: training loss=0.00, validation loss=0.00
Step 221/243: training loss=0.00
Step 222/243: training loss=0.00
Step 223/243: training loss=0.00
Step 224/243: training loss=0.07
Step 225/243: training loss=0.00
Step 226/2

# 학습된 모델 ID 받기

In [26]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:rowan:recipe-ner:9kgRzukw


# 테스트 데이터 만들기

In [28]:
test_df = recipe_df.loc[91:100]
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
test_messages.append({"role": "user", "content": create_user_message(test_row)})

pprint(test_messages)

[{'content': 'You are a helpful recipe assistant. You are to extract the '
             'generic ingredients from each of the recipes provided.',
  'role': 'system'},
 {'content': 'Title: Pound Cake\n'
             '\n'
             'Ingredients: ["3 c. plain flour (cake flour may be substituted '
             'for plain flour)", "3 c. sugar", "1/2 tsp. baking powder", "3 '
             'dashes of salt", "5 eggs", "2 sticks margarine", "1/2 c. butter '
             'flavor Crisco", "1 c. sweet milk", "1 to 2 Tbsp. flavoring '
             '(lemon, vanilla or almond or a combination of the 3)"]\n'
             '\n'
             'Generic ingredients: ',
  'role': 'user'}]


# 호출

gpt-3.5-turboo와 같은 기존 모델이 아닌, 새로 학습한 모델(fine_tuned_model_id)을 주어 호출한다.

In [29]:
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

["flour", "sugar", "baking powder", "salt", "eggs", "margarine", "butter", "sweet milk", "flavoring"]
